In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import data
import shutil
import math
from datetime import datetime
from tensorflow.python.feature_column import feature_column

from tensorflow.contrib.learn import learn_runner
from tensorflow.contrib.learn import make_export_strategy

print(tf.__version__)

/Users/khalidsalama/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


1.4.0


## Steps to use the TF Experiment APIs
1. Define dataset **metadata**
2. Define **data input function** to read the data from .tfrecord files + **feature processing**
3. Create TF **feature columns** based on metadata + **extended feature columns**
4. Define an **estimator** (LinearCombinedDNNClassifier) with the required **feature columns (wide/deep) & parameters**
7. Run an **Experiment** with **learn_runner** to train, evaluate, and export the model
8. **Evaluate** the model using test data
9. Perform **predictions** & **serving** the exported model

In [2]:
MODEL_NAME = 'class-model-01'

TRAIN_DATA_FILES_PATTERN = 'data/train-*.tfrecords'
VALID_DATA_FILES_PATTERN = 'data/valid-*.tfrecords'
TEST_DATA_FILES_PATTERN = 'data/test-*.tfrecords'

RESUME_TRAINING = False
PROCESS_FEATURES = True
EXTEND_FEATURE_COLUMNS = True
MULTI_THREADING = True

## 1. Define Dataset Metadata
* tf.example feature names and defaults
* Numeric and categorical feature names
* Target feature name
* Target feature labels
* Unused features

In [3]:
HEADER = ['key','x','y','alpha','beta','target']
HEADER_DEFAULTS = [[0], [0.0], [0.0], ['NA'], ['NA'], [0.0]]

NUMERIC_FEATURE_NAMES = ['x', 'y']  

CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY = {'alpha':['ax01', 'ax02'], 'beta':['bx01', 'bx02']}
CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.keys())

FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES

TARGET_NAME = 'target'

TARGET_LABELS = ['positive', 'negative']

UNUSED_FEATURE_NAMES = list(set(HEADER) - set(FEATURE_NAMES) - {TARGET_NAME})

print("Header: {}".format(HEADER))
print("Numeric Features: {}".format(NUMERIC_FEATURE_NAMES))
print("Categorical Features: {}".format(CATEGORICAL_FEATURE_NAMES))
print("Target: {} - labels: {}".format(TARGET_NAME, TARGET_LABELS))
print("Unused Features: {}".format(UNUSED_FEATURE_NAMES))

Header: ['key', 'x', 'y', 'alpha', 'beta', 'target']
Numeric Features: ['x', 'y']
Categorical Features: ['alpha', 'beta']
Target: target - labels: ['positive', 'negative']
Unused Features: ['key']


## 2. Define Data Input Function
* Input .tfrecords files name pattern
* Use TF Dataset APIs to read and process the data
* Parse tf.exmaples to feature tensors
* Apply feature processing
* Return (features, target) tensors

### a. parsing and processing logic

In [4]:
def parse_tf_example(example_proto):

    feature_spec = {}

    for feature_name in NUMERIC_FEATURE_NAMES:
        feature_spec[feature_name] = tf.FixedLenFeature(shape=(), dtype=tf.float32)
    
    for feature_name in CATEGORICAL_FEATURE_NAMES:
        feature_spec[feature_name] = tf.FixedLenFeature(shape=(), dtype=tf.string)
    
    feature_spec[TARGET_NAME] = tf.FixedLenFeature(shape=(), dtype=tf.string)

    parsed_features = tf.parse_example(serialized=example_proto, features=feature_spec)
    
    target = parsed_features.pop(TARGET_NAME)
    
    return parsed_features, target


def process_features(features):

    features["x_2"] = tf.square(features['x'])
    features["y_2"] = tf.square(features['y'])
    features["xy"] = tf.multiply(features['x'], features['y']) # features['x'] * features['y']
    features['dist_xy'] =  tf.sqrt(tf.squared_difference(features['x'],features['y']))
    
    return features

### b. data pipeline input function

In [5]:
def tfrecods_input_fn(files_name_pattern, mode=tf.estimator.ModeKeys.EVAL, 
                 num_epochs=None, 
                 batch_size=200):
    
    shuffle = True if mode == tf.estimator.ModeKeys.TRAIN else False
    
    print("")
    print("* data input_fn:")
    print("================")
    print("Input file(s): {}".format(files_name_pattern))
    print("Batch size: {}".format(batch_size))
    print("Epoch Count: {}".format(num_epochs))
    print("Mode: {}".format(mode))
    print("Shuffle: {}".format(shuffle))
    print("================")
    print("")

    file_names = tf.matching_files(files_name_pattern)
    dataset = data.TFRecordDataset(filenames=file_names)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda tf_example: parse_tf_example(tf_example))
    
    if PROCESS_FEATURES:
        dataset = dataset.map(lambda features, target: (process_features(features), target))
        
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()
    
    features, target = iterator.get_next()
    return features, target

In [6]:
features, target = tfrecods_input_fn(files_name_pattern="")
print("Feature read from TFRecords: {}".format(list(features.keys())))
print("Target read from TFRecords: {}".format(target))


* data input_fn:
Input file(s): 
Batch size: 200
Epoch Count: None
Mode: eval
Shuffle: False

Feature read from TFRecords: ['alpha', 'beta', 'x', 'y', 'x_2', 'y_2', 'xy', 'dist_xy']
Target read from TFRecords: Tensor("IteratorGetNext:8", shape=(?,), dtype=string)


## 3. Define Feature Columns

In [7]:
def extend_feature_columns(feature_columns, hparams):
    
    num_buckets = hparams.num_buckets
    embedding_size = hparams.embedding_size

    buckets = np.linspace(-3, 3, num_buckets).tolist()

    alpha_X_beta = tf.feature_column.crossed_column(
            [feature_columns['alpha'], feature_columns['beta']], 4)

    x_bucketized = tf.feature_column.bucketized_column(
            feature_columns['x'], boundaries=buckets)

    y_bucketized = tf.feature_column.bucketized_column(
            feature_columns['y'], boundaries=buckets)

    x_bucketized_X_y_bucketized = tf.feature_column.crossed_column(
           [x_bucketized, y_bucketized], num_buckets**2)

    x_bucketized_X_y_bucketized_embedded = tf.feature_column.embedding_column(
            x_bucketized_X_y_bucketized, dimension=embedding_size)


    feature_columns['alpha_X_beta'] = alpha_X_beta
    feature_columns['x_bucketized_X_y_bucketized'] = x_bucketized_X_y_bucketized
    feature_columns['x_bucketized_X_y_bucketized_embedded'] = x_bucketized_X_y_bucketized_embedded
    
    return feature_columns
    

def get_feature_columns(hparams):
    
    CONSTRUCTED_NUMERIC_FEATURES_NAMES = ['x_2', 'y_2', 'xy', 'dist_xy']
    all_numeric_feature_names = NUMERIC_FEATURE_NAMES.copy() 
    
    if PROCESS_FEATURES:
        all_numeric_feature_names += CONSTRUCTED_NUMERIC_FEATURES_NAMES

    numeric_columns = {feature_name: tf.feature_column.numeric_column(feature_name)
                       for feature_name in all_numeric_feature_names}

    categorical_column_with_vocabulary = \
        {item[0]: tf.feature_column.categorical_column_with_vocabulary_list(item[0], item[1])
         for item in CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.items()}
        
    feature_columns = {}

    if numeric_columns is not None:
        feature_columns.update(numeric_columns)

    if categorical_column_with_vocabulary is not None:
        feature_columns.update(categorical_column_with_vocabulary)
    
    if EXTEND_FEATURE_COLUMNS:
        feature_columns = extend_feature_columns(feature_columns, hparams)
        
    return feature_columns

feature_columns = get_feature_columns(tf.contrib.training.HParams(num_buckets=5,embedding_size=3))
print("Feature Columns: {}".format(feature_columns))

Feature Columns: {'x': _NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'y': _NumericColumn(key='y', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'x_2': _NumericColumn(key='x_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'y_2': _NumericColumn(key='y_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'xy': _NumericColumn(key='xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'dist_xy': _NumericColumn(key='dist_xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'alpha': _VocabularyListCategoricalColumn(key='alpha', vocabulary_list=('ax01', 'ax02'), dtype=tf.string, default_value=-1, num_oov_buckets=0), 'beta': _VocabularyListCategoricalColumn(key='beta', vocabulary_list=('bx01', 'bx02'), dtype=tf.string, default_value=-1, num_oov_buckets=0), 'alpha_X_beta': _CrossedColumn(keys=(_VocabularyListCategoricalColumn(key=

## 4. Define an Estimator Creation Function

### a. Get wide and deep feature columns
* dense columns = numeric columns + embedding columns
* categorical columns = vocabolary list columns + bucketized columns
* sparse columns = hashed categorical columns + crossed columns
* categorical columns => indicator columns
* deep columns = dense columns + indicator columns
* wide columns = categorical columns + sparse columns

In [8]:
def get_wide_deep_columns():
    
    feature_columns = list(get_feature_columns(hparams).values())
    
    dense_columns = list(
        filter(lambda column: isinstance(column, feature_column._NumericColumn) |
                              isinstance(column, feature_column._EmbeddingColumn),
               feature_columns
        )
    )

    categorical_columns = list(
        filter(lambda column: isinstance(column, feature_column._VocabularyListCategoricalColumn) |
                              isinstance(column, feature_column._BucketizedColumn),
                   feature_columns)
    )
    
    sparse_columns = list(
        filter(lambda column: isinstance(column,feature_column._HashedCategoricalColumn) |
                              isinstance(column, feature_column._CrossedColumn),
               feature_columns)
    )

    indicator_columns = list(
            map(lambda column: tf.feature_column.indicator_column(column),
                categorical_columns)
    )
    
    deep_feature_columns = dense_columns + indicator_columns
    wide_feature_columns = categorical_columns + sparse_columns
    
    return wide_feature_columns, deep_feature_columns
    

In [9]:
def create_estimator(run_config, hparams, print_desc=False):
    
    wide_feature_columns, deep_feature_columns = get_wide_deep_columns()
    
    estimator = tf.estimator.DNNLinearCombinedClassifier(
        
        n_classes= len(TARGET_LABELS),
        label_vocabulary=TARGET_LABELS,
        
        dnn_feature_columns = deep_feature_columns,
        linear_feature_columns = wide_feature_columns,
        
        dnn_hidden_units= hparams.hidden_units,
        
        dnn_optimizer= tf.train.AdamOptimizer(),
        
        dnn_activation_fn= tf.nn.elu,
        dnn_dropout= hparams.dropout_prob,
        
        config= run_config
    )
    
    if print_desc:
        print("")
        print("*Estimator Type:")
        print("================")
        print(type(estimator))
        print("")
        print("*deep columns:")
        print("==============")
        print(deep_feature_columns)
        print("")
        print("wide columns:")
        print("=============")
        print(wide_feature_columns)
        print("")
    
    return estimator

## 5. Run Experiment

### a. Define Experiment Function

In [10]:
def generate_experiment_fn(**experiment_args):

    def _experiment_fn(run_config, hparams):

        train_input_fn = lambda: tfrecods_input_fn(
            TRAIN_DATA_FILES_PATTERN,
            mode = tf.contrib.learn.ModeKeys.TRAIN,
            num_epochs=hparams.num_epochs,
            batch_size=hparams.batch_size
        )

        eval_input_fn = lambda: tfrecods_input_fn(
            VALID_DATA_FILES_PATTERN,
            mode=tf.contrib.learn.ModeKeys.EVAL,
            num_epochs=1,
            batch_size=hparams.batch_size
        )

        estimator = create_estimator(run_config, hparams, True)

        return tf.contrib.learn.Experiment(
            estimator,
            train_input_fn=train_input_fn,
            eval_input_fn=eval_input_fn,
            eval_steps=None,
            **experiment_args
        )

    return _experiment_fn

### b. Set HParam and RunConfig

In [11]:
TRAIN_SIZE = 12000
NUM_EPOCHS = 1000
BATCH_SIZE = 500
NUM_EVAL = 10
CHECKPOINT_STEPS = int((TRAIN_SIZE/BATCH_SIZE) * (NUM_EPOCHS/NUM_EVAL))

hparams  = tf.contrib.training.HParams(
    num_epochs = NUM_EPOCHS,
    batch_size = BATCH_SIZE,
    hidden_units=[16, 12, 8],
    num_buckets = 6,
    embedding_size = 3,
    dropout_prob = 0.001)

model_dir = 'trained_models/{}'.format(MODEL_NAME)

run_config = tf.contrib.learn.RunConfig(
    save_checkpoints_steps=CHECKPOINT_STEPS,
    tf_random_seed=19830610,
    model_dir=model_dir
)

print(hparams)
print("Model Directory:", run_config.model_dir)
print("")
print("Dataset Size:", TRAIN_SIZE)
print("Batch Size:", BATCH_SIZE)
print("Steps per Epoch:",TRAIN_SIZE/BATCH_SIZE)
print("Total Steps:", (TRAIN_SIZE/BATCH_SIZE)*NUM_EPOCHS)
print("Required Evaluation Steps:", NUM_EVAL) 
print("That is 1 evaluation step after each",NUM_EPOCHS/NUM_EVAL," epochs")
print("Save Checkpoint After",CHECKPOINT_STEPS,"steps")

[('batch_size', 500), ('dropout_prob', 0.001), ('embedding_size', 3), ('hidden_units', [16, 12, 8]), ('num_buckets', 6), ('num_epochs', 1000)]
Model Directory: trained_models/class-model-01

Dataset Size: 12000
Batch Size: 500
Steps per Epoch: 24.0
Total Steps: 24000.0
Required Evaluation Steps: 10
That is 1 evaluation step after each 100.0  epochs
Save Checkpoint After 2400 steps


### c. Define Serving Function

In [12]:
def csv_serving_input_fn():
    
    SERVING_HEADER = ['x','y','alpha','beta']
    SERVING_HEADER_DEFAULTS = [[0.0], [0.0], ['NA'], ['NA']]

    rows_string_tensor = tf.placeholder(dtype=tf.string,
                                         shape=[None],
                                         name='csv_rows')
    
    receiver_tensor = {'csv_rows': rows_string_tensor}

    row_columns = tf.expand_dims(rows_string_tensor, -1)
    columns = tf.decode_csv(row_columns, record_defaults=SERVING_HEADER_DEFAULTS)
    features = dict(zip(SERVING_HEADER, columns))

    return tf.estimator.export.ServingInputReceiver(
        process_features(features), receiver_tensor)

### d. Run Experiment via learn_runner

In [13]:
if not RESUME_TRAINING:
    print("Removing previous artifacts...")
    shutil.rmtree(model_dir, ignore_errors=True)
else:
    print("Resuming training...") 


tf.logging.set_verbosity(tf.logging.INFO)

time_start = datetime.utcnow() 
print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
print(".......................................") 


learn_runner.run(
    experiment_fn=generate_experiment_fn(

        export_strategies=[make_export_strategy(
            csv_serving_input_fn,
            exports_to_keep=1
        )]
    ),
    run_config=run_config,
    schedule="train_and_evaluate",
    hparams=hparams
)

time_end = datetime.utcnow() 
print(".......................................")
print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
print("")
time_elapsed = time_end - time_start
print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))
    

Removing previous artifacts...
Experiment started at 10:47:36
.......................................
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x121edc320>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': 19830610, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 2400, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'trained_models/class-model-01'}

*Estimator Type:
<class 'tensorflow.python.estimator.canned.dnn_linear_combined.DNNLinearCombinedClassifier'>

*deep columns:
[_NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='y

INFO:tensorflow:Starting evaluation at 2017-11-16-10:48:12
INFO:tensorflow:Restoring parameters from trained_models/class-model-01/model.ckpt-2401
INFO:tensorflow:Finished evaluation at 2017-11-16-10:48:13
INFO:tensorflow:Saving dict for global step 2401: accuracy = 0.948667, accuracy_baseline = 0.5, auc = 0.989728, auc_precision_recall = 0.99096, average_loss = 0.127021, global_step = 2401, label/mean = 0.5, loss = 63.5107, prediction/mean = 0.501595
INFO:tensorflow:Validation (step 2401): accuracy = 0.948667, accuracy_baseline = 0.5, auc = 0.989728, auc_precision_recall = 0.99096, average_loss = 0.127021, label/mean = 0.5, loss = 63.5107, prediction/mean = 0.501595, global_step = 2401
INFO:tensorflow:loss = 43.2125, step = 2401 (6.067 sec)
INFO:tensorflow:global_step/sec: 24.899
INFO:tensorflow:loss = 51.4669, step = 2501 (0.676 sec)
INFO:tensorflow:global_step/sec: 161.76
INFO:tensorflow:loss = 62.8036, step = 2601 (0.619 sec)
INFO:tensorflow:global_step/sec: 157.281
INFO:tensorflow

INFO:tensorflow:loss = 50.571, step = 7901 (0.614 sec)
INFO:tensorflow:global_step/sec: 145.843
INFO:tensorflow:loss = 35.5534, step = 8001 (0.691 sec)
INFO:tensorflow:global_step/sec: 152.191
INFO:tensorflow:loss = 47.7327, step = 8101 (0.653 sec)
INFO:tensorflow:global_step/sec: 158.159
INFO:tensorflow:loss = 55.5154, step = 8201 (0.631 sec)
INFO:tensorflow:global_step/sec: 173.279
INFO:tensorflow:loss = 45.991, step = 8301 (0.576 sec)
INFO:tensorflow:global_step/sec: 177.057
INFO:tensorflow:loss = 47.9003, step = 8401 (0.564 sec)
INFO:tensorflow:global_step/sec: 198.749
INFO:tensorflow:loss = 56.759, step = 8501 (0.504 sec)
INFO:tensorflow:global_step/sec: 182.828
INFO:tensorflow:loss = 68.8786, step = 8601 (0.548 sec)
INFO:tensorflow:global_step/sec: 163.002
INFO:tensorflow:loss = 62.8279, step = 8701 (0.613 sec)
INFO:tensorflow:global_step/sec: 185.434
INFO:tensorflow:loss = 66.1603, step = 8801 (0.539 sec)
INFO:tensorflow:global_step/sec: 136.639
INFO:tensorflow:loss = 53.6692, s

INFO:tensorflow:global_step/sec: 188.945
INFO:tensorflow:loss = 53.7547, step = 14201 (0.527 sec)
INFO:tensorflow:global_step/sec: 202.411
INFO:tensorflow:loss = 54.4652, step = 14301 (0.494 sec)
INFO:tensorflow:Saving checkpoints for 14401 into trained_models/class-model-01/model.ckpt.
INFO:tensorflow:global_step/sec: 49.3556

* data input_fn:
Input file(s): data/valid-*.tfrecords
Batch size: 500
Epoch Count: 1
Mode: eval
Shuffle: False

INFO:tensorflow:Starting evaluation at 2017-11-16-10:49:49
INFO:tensorflow:Restoring parameters from trained_models/class-model-01/model.ckpt-14401
INFO:tensorflow:Finished evaluation at 2017-11-16-10:49:51
INFO:tensorflow:Saving dict for global step 14401: accuracy = 0.953667, accuracy_baseline = 0.5, auc = 0.990981, auc_precision_recall = 0.991663, average_loss = 0.121245, global_step = 14401, label/mean = 0.5, loss = 60.6223, prediction/mean = 0.499904
INFO:tensorflow:Validation (step 14401): accuracy = 0.953667, accuracy_baseline = 0.5, auc = 0.99

INFO:tensorflow:loss = 49.0712, step = 19301 (0.608 sec)
INFO:tensorflow:global_step/sec: 145.232
INFO:tensorflow:loss = 65.2483, step = 19401 (0.692 sec)
INFO:tensorflow:global_step/sec: 171.492
INFO:tensorflow:loss = 41.6333, step = 19501 (0.578 sec)
INFO:tensorflow:global_step/sec: 151.576
INFO:tensorflow:loss = 70.4016, step = 19601 (0.661 sec)
INFO:tensorflow:global_step/sec: 147.299
INFO:tensorflow:loss = 46.5377, step = 19701 (0.678 sec)
INFO:tensorflow:global_step/sec: 190.115
INFO:tensorflow:loss = 46.9032, step = 19801 (0.525 sec)
INFO:tensorflow:global_step/sec: 192.531
INFO:tensorflow:loss = 46.2393, step = 19901 (0.519 sec)
INFO:tensorflow:global_step/sec: 193.22
INFO:tensorflow:loss = 33.4292, step = 20001 (0.517 sec)
INFO:tensorflow:global_step/sec: 196.5
INFO:tensorflow:loss = 51.4318, step = 20101 (0.509 sec)
INFO:tensorflow:global_step/sec: 192.817
INFO:tensorflow:loss = 60.9859, step = 20201 (0.519 sec)
INFO:tensorflow:global_step/sec: 193.571
INFO:tensorflow:loss = 

## 6. Evaluate the Model

In [14]:
TRAIN_SIZE = 12000
VALID_SIZE = 3000
TEST_SIZE = 5000
train_input_fn = lambda: tfrecods_input_fn(files_name_pattern= TRAIN_DATA_FILES_PATTERN, 
                                      mode= tf.estimator.ModeKeys.EVAL,
                                      batch_size= TRAIN_SIZE)

valid_input_fn = lambda: tfrecods_input_fn(files_name_pattern= VALID_DATA_FILES_PATTERN, 
                                      mode= tf.estimator.ModeKeys.EVAL,
                                      batch_size= VALID_SIZE)

test_input_fn = lambda: tfrecods_input_fn(files_name_pattern= TEST_DATA_FILES_PATTERN, 
                                      mode= tf.estimator.ModeKeys.EVAL,
                                      batch_size= TEST_SIZE)

estimator = create_estimator(run_config, hparams)

train_results = estimator.evaluate(input_fn=train_input_fn, steps=1)
print()
print("############################################################################################")
print("# Train Measures: {}".format(train_results))
print("############################################################################################")

valid_results = estimator.evaluate(input_fn=valid_input_fn, steps=1)
print()
print("############################################################################################")
print("# Valid Measures: {}".format(valid_results))
print("############################################################################################")

test_results = estimator.evaluate(input_fn=test_input_fn, steps=1)
print()
print("############################################################################################")
print("# Test Measures: {}".format(test_results))
print("############################################################################################")

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x121edc320>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': 19830610, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 2400, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'trained_models/class-model-01'}

* data input_fn:
Input file(s): data/train-*.tfrecords
Batch size: 12000
Epoch Count: None
Mode: eval
Shuffle: False

INFO:tensorflow:Starting evaluation at 2017-11-16-10:51:11
INFO:tensorflow:Restoring parameters from trained_models/class-model-01/model.ckpt-24000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-11

## 7. Prediction

In [15]:
import itertools

predict_input_fn = lambda: tfrecods_input_fn(files_name_pattern= TEST_DATA_FILES_PATTERN, 
                                      mode= tf.estimator.ModeKeys.PREDICT,
                                      batch_size= 5)


predictions = list(itertools.islice(estimator.predict(input_fn=predict_input_fn),5))

print("")
print("* Predicted Classes: {}".format(list(map(lambda item: item["class_ids"][0]
    ,predictions))))

print("* Predicted Probabilities: {}".format(list(map(lambda item: list(item["probabilities"])
    ,predictions))))


* data input_fn:
Input file(s): data/test-*.tfrecords
Batch size: 5
Epoch Count: None
Mode: infer
Shuffle: False

INFO:tensorflow:Restoring parameters from trained_models/class-model-01/model.ckpt-24000

* Predicted Classes: [0, 1, 1, 0, 1]
* Predicted Probabilities: [[0.98284829, 0.017151766], [0.00044470496, 0.99955529], [0.00017617996, 0.99982387], [0.99743325, 0.0025667532], [0.010059897, 0.98994011]]


## Serving via the Saved Model

In [16]:
import os

export_dir = model_dir +"/export/Servo/"

saved_model_dir = export_dir + "/" + os.listdir(path=export_dir)[-1] 

print(saved_model_dir)

predictor_fn = tf.contrib.predictor.from_saved_model(
    export_dir = saved_model_dir,
    signature_def_key="predict"
)

output = predictor_fn({'csv_rows': ["0.5,1,ax01,bx02", "-0.5,-1,ax02,bx02"]})
print(output)

trained_models/class-model-01/export/Servo//1510829467
INFO:tensorflow:Restoring parameters from b'trained_models/class-model-01/export/Servo//1510829467/variables/variables'
{'logits': array([[ 6.35565805],
       [ 6.37263775]], dtype=float32), 'logistic': array([[ 0.99826616],
       [ 0.99829525]], dtype=float32), 'probabilities': array([[ 0.00173388,  0.99826616],
       [ 0.00170474,  0.99829525]], dtype=float32), 'class_ids': array([[1],
       [1]]), 'classes': array([[b'negative'],
       [b'negative']], dtype=object)}
